In [1]:
import h5py
import scipy.io as scio
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import setting_2 as setting
from random import shuffle
import math
from sklearn.decomposition import PCA
import cv2

In [2]:
NO='38'

data_file =NO  # File name of the data set
data_name = 'X'  # Matrix name of the data set within the data file
label_file='Label_'+NO
no=setting.train_no
train_data_name='./sample/Train_'+str(no)+'_'+NO
print(train_data_name)

./sample/Train_2_38


In [3]:
def patch_data(height_index, width_index):
    height_slice = slice(height_index, height_index + PATCH_SIZE)
    width_slice = slice(width_index, width_index + PATCH_SIZE)
    patches = data_norm[height_slice, width_slice,:]
    return np.array(patches)

In [4]:
# 加载数据
# input_mat = scio.loadmat(os.path.join(setting.train_data_path, data_file + '.mat'))[data_name]
input_mat = h5py.File(os.path.join(setting.train_data_path, data_file + '.mat')) [data_name]
input_mat=np.array(input_mat).transpose(2,1,0)
label_map = cv2.imread(os.path.join(setting.train_label_path, label_file + '.png'), cv2.IMREAD_GRAYSCALE)

print(input_mat.shape)
print(label_map.shape)
print(np.unique(label_map))

(512, 512, 204)
(512, 512)
[0 1 2]


In [5]:
#打印参数 
[HEIGHT, WIDTH, BAND] = input_mat.shape
print('HEIGHT=',HEIGHT, 'WIDTH=',WIDTH, 'BAND=',BAND)
PATCH_SIZE = setting.PATCH_SIZE
print('PATCH_SIZE=',PATCH_SIZE)
PATCH_IDX = int((PATCH_SIZE - 1) / 2)
print('PATCH_IDX=', PATCH_IDX)
CLASS_NUM = setting.CLASS_NUM
print('CLASS_NUM=',CLASS_NUM)
DTYPE=setting.DTYPE
print('DTYPE:', DTYPE)

HEIGHT= 512 WIDTH= 512 BAND= 204
PATCH_SIZE= 7
PATCH_IDX= 3
CLASS_NUM= 3
DTYPE: float64


In [6]:
input_data=input_mat
print('data max:',np.max(input_data))
print('data min:',np.min(input_data))

data max: 1.218181848526001
data min: 0.06063193827867508


In [7]:
# pad
input_pad = np.zeros(((HEIGHT + PATCH_SIZE - 1), (WIDTH + PATCH_SIZE - 1), input_data.shape[2]))
input_pad[PATCH_IDX:(HEIGHT + PATCH_IDX), PATCH_IDX:(WIDTH + PATCH_IDX), :] = input_data[:]
input_pad[PATCH_IDX:(HEIGHT + PATCH_IDX), :PATCH_IDX, :] = input_data[:, (PATCH_IDX - 1)::-1, :]
input_pad[PATCH_IDX:(HEIGHT + PATCH_IDX), (WIDTH + PATCH_IDX):, :] = input_data[:, :(WIDTH - PATCH_IDX - 1):-1, :]
input_pad[:PATCH_IDX, :, :] = input_pad[(PATCH_IDX * 2 - 1):(PATCH_IDX - 1):-1, :, :]
input_pad[(HEIGHT + PATCH_IDX):, :, :] = input_pad[(HEIGHT + PATCH_IDX - 1):(HEIGHT - 1):-1, :, :]
print('size after pad:',input_pad.shape,'\nmax:',np.max(input_pad),'\nmin:',np.min(input_pad))

size after pad: (518, 518, 204) 
max: 1.218181848526001 
min: 0.06063193827867508


In [8]:
data_norm=input_pad.astype(DTYPE)
print(data_norm.shape)
print('max:',np.max(data_norm),'\nmin:',np.min(data_norm))

(518, 518, 204)
max: 1.218181848526001 
min: 0.06063193827867508


In [9]:
# 按类别归整样本
CLASSES = []
POSITION=[]
for i in range(CLASS_NUM):
    CLASSES.append([])
    POSITION.append([])
for i in range(HEIGHT):
    for j in range(WIDTH):
        temp_y = label_map[i, j]
        if temp_y != 0:
            temp_x = patch_data(i, j)
            CLASSES[temp_y].append(temp_x)
            POSITION[temp_y].append([i,j])
for i in range(CLASS_NUM):
    print ('CLASS',str(i) + '\t',len(CLASSES[i]))

CLASS 0	 0
CLASS 1	 1357
CLASS 2	 1241


In [10]:
TRAIN_PATCH, TRAIN_LABELS = [], []
# class_name=setting.class_name
# for c in class_name:  # for each class
for c in range(CLASS_NUM):
    print(c)
    num=len(CLASSES[c])
    if num==0:
        print('this class is empty')
        continue
    randIdx = np.arange(num)
    shuffle(randIdx)  # Randomly shuffle patches in the class
    current_class_data = np.array(CLASSES[c],dtype = DTYPE)
    rand_current_class_data = current_class_data[randIdx,:,:,:]
    print('current class data size:', rand_current_class_data.shape)
    if c==1:
        TRAIN_LABELS.extend(np.full(num, 0, dtype=int))
    elif c==2:
        TRAIN_LABELS.extend(np.full(num, 1, dtype=int))
    TRAIN_PATCH.extend(rand_current_class_data)
#     TRAIN_LABELS.extend(np.full(num, c, dtype=int))
TRAIN_PATCH = np.array(TRAIN_PATCH, dtype=DTYPE)
print ('\nTotal num of Training patches: %d\n' % len(TRAIN_PATCH))
print('label class name:',np.unique(TRAIN_LABELS))

0
this class is empty
1
current class data size: (1357, 7, 7, 204)
2
current class data size: (1241, 7, 7, 204)

Total num of Training patches: 2598

label class name: [0 1]


In [11]:
# save files
# 1. Training data
file_name = train_data_name +'_'+ str(PATCH_SIZE) +'_'+ str(DTYPE) +'.h5'
print ('Writing: ' + file_name)
with h5py.File(file_name, 'w') as file:
    file.create_dataset('train_patch', data=TRAIN_PATCH)
    file.create_dataset('train_labels', data=TRAIN_LABELS, dtype='i8')
print ('Successfully save training data set!')

Writing: ./sample/Train_2_38_7_float64.h5
Successfully save training data set!
